In [ ]:
import numpy as np
import pandas as pd

# pd.set_option('display.height', 1000)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.set_option("max_colwidth", 200)
from IPython.core.display import HTML, display

display(HTML("<style>.container { width:100% !important; }</style>"))


import os

os.environ["R_HOME"] = "/root/miniconda3/envs/R/lib/R"

import rpy2.robjects as objects
from rpy2.robjects.packages import importr

base = importr("base")
r_pROC = importr("pROC")
base._libPaths()[0]


def df_stats(df):
    from tabulate import tabulate

    print("\n***** Shape: ", df.shape, " *****\n")

    columns_list = df.columns.values.tolist()
    isnull_list = df.isnull().sum().values.tolist()
    isunique_list = df.nunique().values.tolist()
    dtypes_list = df.dtypes.tolist()

    list_stat_val = list(zip(columns_list, isnull_list, isunique_list, dtypes_list))
    df_stat_val = pd.DataFrame(
        list_stat_val, columns=["Name", "Null", "Unique", "Dtypes"]
    )
    print(tabulate(df_stat_val, headers="keys", tablefmt="psql"))
    return df.head()

In [ ]:
df_ifr = pd.read_csv(
    "../CathAI/data/DeepCORO/CathReport_MHI_Merged/2017-2021_CathReport_first_value_in_interval_IFR.csv"
)

In [ ]:
## Define column where IFR was performed
import pandas as pd


def create_new_col(df):
    # List of column names to be checked
    col_names = [
        "D2",
        "D3",
        "RVG1",
        "RVG2",
        "S1",
        "Saphène ou mammaire",
        "bx",
        "diagonal",
        "dist_lad",
        "dist_lcx",
        "dist_rca",
        "lad",
        "lcx",
        "leftmain",
        "lvp",
        "marg_d",
        "mid_lad",
        "mid_rca",
        "om1",
        "om2",
        "om3",
        "pda",
        "posterolateral",
        "prox_rca",
    ]

    # Iterating over rows using DataFrame.apply() and lambda function
    df["ifr_performed"] = df.apply(
        lambda row: ", ".join([col for col in col_names if row[col] != -1.0]), axis=1
    )
    return df

In [ ]:
df = create_new_col(df_ifr)
df["contains_lad"] = df["ifr_performed"].str.contains("lad")
df["contains_lcx"] = df["ifr_performed"].str.contains("lcx")
df["contains_rca"] = df["ifr_performed"].str.contains("rca|pda|posterolateral")

In [ ]:
display(df_ifr.head(n=1))

In [ ]:
### Merge with DICOMS Extracted

df_dicoms = pd.read_csv(
    "../CathAI/data/DeepCORO/EXAMS_Extracted/2017-2021_dicom_extracted_mod.csv"
)

In [ ]:
# Identify common columns in df and df_dicoms, excluding 'path'
common_cols = [col for col in df.columns if col in df_dicoms.columns and col != "path"]

# Drop common columns from df_dicoms
df_dicoms_dropped = df_dicoms.drop(columns=common_cols)

# Perform left merge
df_merged = pd.merge(
    df, df_dicoms_dropped, left_on="DICOMPath", right_on="path", how="left"
)

In [ ]:
### Merge with views
df_angle_object = pd.read_csv(
    "../CathAI/data/DeepCORO/CATHAI_Extracted_Concatenated/DeepCORO_df_angle_object_dicom_2017-2021.csv"
)
### Drop Unnamed columns
df_angle_object = df_angle_object.loc[
    :, ~df_angle_object.columns.str.contains("^Unnamed")
]

In [ ]:
# Perform left merge
df_merged_angle_object = pd.merge(df_merged, df_angle_object, on="path", how="left")

In [ ]:
display(df_merged_angle_object.object_value.value_counts())

In [ ]:
display(df_merged_angle_object.contains_lad.value_counts())
display(df_merged_angle_object.contains_lcx.value_counts())
display(df_merged_angle_object.contains_rca.value_counts())

In [ ]:
df_filtered = df_merged_angle_object[
    (df_merged_angle_object["contains_lad"] == True)
    | (df_merged_angle_object["contains_lcx"] == True)
    | (df_merged_angle_object["contains_rca"] == True)
]

In [ ]:
#1425 examens avec IFR dans IVA, LCX ou RCA
display(df_f.grou

In [ ]:
df_f.to_csv("data/df_f.csv", index=False)
df_merged_angle_object.to_csv("data/df_merged_angle_object_IFR.csv", index=False)

Sure, here's a markdown table based on the labels' definitions:

| Label | Name         | 'primary' range | 'secondary' range |
|-------|--------------|-----------------|-------------------|
| 0     | RAO Cranial  | -60 to -15      | 15 to 50          |
| 1     | AP Cranial   | -15 to 15       | 15 to 50          |
| 2     | LAO Cranial  | 15 to 60        | 15 to 50          |
| 3     | RAO Straight | -60 to -15      | -15 to 15         |
| 4     | AP           | -15 to 15       | -15 to 15         |
| 5     | LAO Straight | 15 to 60        | -15 to 15         |
| 6     | RAO Caudal   | -60 to -15      | -50 to -15        |
| 7     | AP Caudal    | -15 to 15       | -50 to -15        |
| 8     | LAO Caudal   | 15 to 60        | -50 to -15        |
| 9     | LAO Lateral  | -110 to -70     | -15 to 15         |
| 10    | RAO Lateral  | 70 to 110       | -15 to 15         |
| 11    | Other        | Not applicable  | Not applicable    |

Note: The ranges in the 'primary' and 'secondary' columns are inclusive.

## Generate videos

In [ ]:
import numpy as np
import pandas as pd

# pd.set_option('display.height', 1000)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.set_option("max_colwidth", 200)
from IPython.core.display import HTML, display

display(HTML("<style>.container { width:100% !important; }</style>"))

from downloadAvi import extract_avi_metadata as avi_meta

In [ ]:
df_f = pd.read_csv("data/df_f.csv")
# df_merged_angle_object = pd.read_csv('data/df_merged_angle_object_IFR.csv')

In [ ]:
avi_meta.extract_avi_and_metadata(
    "data/df_f.csv", data_type="ANGIO", destinationFolder="deepIFR/", dataFolder="data/"
)

In [ ]:
import pydicom as dicom
from pydicom.uid import UID, generate_uid

dataset = dicom.dcmread(df_f.path.iloc[1], force=True)

In [ ]:
display(dataset)